## Rfam pull

In [4]:
import os

def name(idx):
    return "RF{}.fa".format(str(idx).zfill(5))

def link(idx):
    return "https://ftp.ebi.ac.uk/pub/databases/Rfam/CURRENT/fasta_files/{}.gz".format(name(idx))

for i in range(4300, 4301):
    os.system(f"wget -P /Users/yvesmartin/data/rfam {link(i)}")
    os.system(f"gunzip /Users/yvesmartin/data/rfam/{name(i)}.gz")
    os.system(f"rm /Users/yvesmartin/data/rfam/{name(i)}.gz")

--2023-12-12 11:20:40--  https://ftp.ebi.ac.uk/pub/databases/Rfam/CURRENT/fasta_files/RF04300.fa.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 629 [application/x-gzip]
Saving to: ‘/Users/yvesmartin/data/rfam/RF04300.fa.gz’

     0K                                                       100%  200M=0s

2023-12-12 11:20:40 (200 MB/s) - ‘/Users/yvesmartin/data/rfam/RF04300.fa.gz’ saved [629/629]

rm: /Users/yvesmartin/data/rfam/RF04300.fa.gz: No such file or directory


In [5]:
for f in os.listdir("/Users/yvesmartin/data/rfam"):
    if f.endswith(".gz.1"):
        os.system(f"rm /Users/yvesmartin/data/rfam/{f}")

## Make it a dataset

In [3]:
import os

def fasta_parser(f):
    while line:=f.readline():
        if line.startswith(">"):
            ref = line[1:].split()[0]
            desc = ' '.join(line.split()[1:]).strip()
        else:
            seq = line.strip()
            yield ref, desc, seq
            
fams, refs, descs, seqs = [], [], [], []
refs_count = {}
for f in os.listdir("/Users/yvesmartin/data/rfam"):
    if f.endswith(".fa"):
        with open("/Users/yvesmartin/data/rfam/{}".format(f), "r") as f:
            for ref, desc, seq in fasta_parser(f):
                fams.append(os.path.basename(f.name).replace(".fa", ""))
                if ref in refs_count:
                    refs.append("{}_{}".format(ref, refs_count[ref]))
                else:
                    refs.append(ref)
                descs.append(desc)
                seqs.append(seq)
                
                refs_count[ref] = refs_count.get(ref, 0) + 1
                
import pandas as pd
df = pd.DataFrame({"family": fams, "reference": refs, "description": descs, "sequence": seqs}).set_index("reference")
df.to_json("data/rfam.json", orient="index", indent=2)